# Astra DB for Customer Service Agent

Learn how to use your Astra DB database with AstraPy.

In this quickstart, you'll create a vector collection, store a few documents on it, and run **vector searches** on it.

_Prerequisites:_ Make sure you have an Astra DB instance and get ready to supply the corresponding *Token* and the *API Endpoint*
(read more [here](https://docs.datastax.com/en/astra/home/astra.html)).

## Setup

In [2]:
!pip install --quiet --upgrade astrapy openai tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, bu

### Import needed libraries

In [3]:
import os, json
from getpass import getpass

from astrapy.db import AstraDB, AstraDBCollection

### Provide database credentials

These are the connection parameters on your Astra dashboard. Example values:

- API Endpoint: `https://01234567-89ab-cdef-0123-456789abcdef-us-east1.apps.astra.datastax.com`
- Token: `AstraCS:6gBhNmsk135...`


In [12]:
ASTRA_DB_API_ENDPOINT = input("ASTRA_DB_API_ENDPOINT = ")
ASTRA_DB_APPLICATION_TOKEN = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

ASTRA_DB_API_ENDPOINT = https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com
ASTRA_DB_APPLICATION_TOKEN = ··········


## Create a collection

### Create the client

In [13]:
astra_db = AstraDB(
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

# Carregando Reviews

In [6]:
data = [{
    "source": "",
    "agent": "service_charge",
    "title" : 'Comgás | Ligação do Fogão com o Gás',
    "content" : """Prezada equipe da Comgás,
Escrevo esta reclamação formal para expressar minha insatisfação em relação à cobrança de R$300,00 para a instalação da mangueira de gás para o meu novo fogão, que já possui conversão para gás encanado e nunca foi utilizado. Importante dizer que não foi solicitado a instalação do gás no apartamento e vocês estão cobrando o serviço de instalação da mangueira no fogão até o gás.
Entendo a necessidade de procedimentos de segurança e instalação profissional, porém, considerando que meu fogão já possui a adaptação necessária, a cobrança mencionada parece injustificada. Gostaria de esclarecimentos sobre o motivo dessa taxa e solicito uma revisão dessa cobrança, levando em consideração as circunstâncias específicas do meu caso.
Agradeço a atenção e espero uma rápida resolução para esta questão.
Atenciosamente,
******* Silva Machado
*******"""
},
        {
            "source": "",
            "agent": "invoice_charge",
            "title": 'Cobrança abusiva',
            "content": """No começo do mês de Janeiro, recebi minha fatura (que tem uma média de no máximo $56) no valor de $450 aproximadamente. Entrei em contato com a Comgás pelo chat e pedi a revisão da fatura. Durante o contato, me foi recomendado pagar essa fatura para não gerar juros e confirmado que ao identificar o problema, meu dinheiro seria devolvido. Pois bem, paguei a fatura e a Comgás enviou um técnico pra fazer a troca do medidor. No dia seguinte, foi constatado que realmente houve um engano no valor cobrado e disseram que o valor correto era de $99, o dobro do que eu costumo pagar. No mês de dezembro não houve nenhum aumento de consumo, na verdade houve uma redução, pois estava muito calor e evitamos usar o forno, fora que viajamos por alguns dias. Pedi novamente a revisão dessa fatura, pois não utilizei o produto e não acho justo ter que pagar esse valor abusivo. Fora a isso, perguntei sobre o reembolso e me informaram que não devolvem o dinheiro, o mesmo fica retido como crédito para as próximas faturas. Não quero crédito, preciso do reembolso, pois não é um valor baixo e está fazendo falta. Hoje recebi retorno da Comgás e disseram que o valor está sim correto dessa fatura, ficou a palavra deles contra a minha, não concordo com essa cobrança do dobro do valor. Moram apenas 2 pessoas na minha casa e a única coisa ligada no gás é o fogão. NÃO UTILIZEI ESSE SERVIÇO E NÃO É JUSTO ME COBRAR ESSE VALOR."""
        },
        {
            "source": "https://www.reclameaqui.com.br/comgas/conta-de-gas-no-valor-de-1153-00_l-YHdz0EejQChl3t/",
            "agent": "invoice_charge",
            "title": 'Conta de gás no valor de 1153,00',
            "content": """Boa tarde
Minha conta de gás do mês de janeiro veio um valor exorbitante de 1153,00 entrei em
Contato com a empresa me disseram que estava correto quero entender como uma residência aonde moram um adulto e uma
Criança pode gasta mais de 1000 por mês sendo q nos meses anteriores nunca chegou nesse valor e a segunda vez que essa empresa faz isso o pior que os especialistas não tem qualificações nenhuma pq qualquer pessoa em sã consciência ia saber q não existe um gasto desse e só analisar minhas contas anteriores a conta estava em débito automático minha conta ficou negativa"""
        },
        {
            "source": "https://www.reclameaqui.com.br/comgas/leitura-de-consumo-de-gas-incorrera_OE7gvTuPWiL2Y0J0/",
            "agent": "invoice_charge",
            "title": "Leitura de consumo de gás incorrera",
            "content": """Boa tarde!

Estou abrindo esta reclamação para solicitar a releitura do meu consumo de gás, pois após comparação das leituras efetuadas nos dois últimos meses (Nov e Dez/23), somado a inspeção do que é apresentado em meu medidor, acredito que a leitura do meu consumo esteja incorreta (conforme evidências em anexo).

Na fatura de Dezembro foi efetuada uma leitura atual de 255, porém em meu medidor, consta 225.

Conforme solicitação recebida em contato com a central de relacionamento da comgás, tirei duas fotos com intervalo de 10min uma da outra e ambas registram o mesmo valor.

Gostaria de pedir o apoio de vocês para entender o que houve e solucionar o meu caso, pois não utilizei o gás no mês de dezembro e recebi uma cobrança de um valor absurdo.


Obrigada!"""
        }
        ]

In [7]:
import os, json
from getpass import getpass
OPENAI_API_KEY = getpass("OPENAI_API_KEY = ")

OPENAI_API_KEY = ··········


In [ ]:
# Inicializando a OpenAI
from openai import OpenAI
client = OpenAI(api_key = OPENAI_API_KEY)

def get_embedding_openai(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model, dimensions=3072).data[0].embedding

embedding = get_embedding_openai("My text to test embeddings")
print(len(embedding))
print(embedding[:5])

In [14]:
astra_db = AstraDB(
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)
collection_name="customer_service_inbox"
# collection = astra_db.create_collection(collection_name,
#                                         dimension=3072
#                                        )

In [16]:
collection = AstraDBCollection(collection_name, astra_db = astra_db)

In [ ]:
collection.delete_many({})

In [27]:
%time
#Load inbox items dataset to Astra
from tqdm import tqdm
docs = []
count = 0
for index, row in tqdm(enumerate(data), desc=f'Loading data'):
    count += 1
    docs.append({
        "_id": f"review_{index}",
        "agent": row["agent"],
        "title": row["title"],
        "content": row["content"],
        "metadata" : {
            "source": row["source"],
        },
        "$vector": get_embedding_openai(row["content"]),
    })

    # 20 is the max limit for Astrapy
    if len(docs) % 10 == 0:
        response = collection.insert_many(docs)
        docs.clear()

response = collection.insert_many(docs)
print(f"{count} loaded - final")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


Loading data: 4it [00:00,  4.38it/s]


4 loaded - final


In [29]:
def findSimilar(emb):
    docs = collection.vector_find(
        emb,
        limit=1,
        fields=["$vector", "title","agent"],
    )

    return docs[0]["title"], docs[0]["$similarity"], docs[0]["_id"], docs[0]["agent"]

In [31]:
# https://www.reclameaqui.com.br/comgas/cobranca_QyWwSL4igUfP2pb8/
new_item = """A Congas está me cobrando uma parcela de instalação que vencia em 10/Outubro de 2023. Ocorre que paguei antecipadamente no dia 02/10. Ja entrei no site e fiz 2 chamados inclusive enviando o comprovante do pagamento, mas não recebo retorno e CONTINUAM COBRANDO.
Esta cobrança é [Editado pelo Reclame Aqui].
Se é que alguém da Congas vai entrar em contato, prefiro que seja por email ou WhatsApp."""

emb = get_embedding_openai(new_item)

titleSim, similarity, simId, AIAgent = findSimilar(emb)


print(f"ID item similar: {simId}")
print(f"Título similar: {titleSim}")
print(f"Similaridade: {similarity}")
print(f"Agente: {AIAgent}")

ID item similar: review_1
Título similar: Cobrança abusiva
Similaridade: 0.8102504
Agente: invoice_charge


### Generating Service Orders

The `create_collection` method results in a new collection on your database.

In [ ]:
collection = astra_db.create_collection("service_orders")

In [ ]:
collection.delete_many({})

## Insert documents

In [ ]:
import json
from datetime import datetime, timedelta
import uuid
import random

# https://www.reclameaqui.com.br/comgas/cobranca_qmOi__ltdYULQeK0/
orders = [
    {
      "_id": "007910599869",
      "customer_id": 'f08a6894-1863-491d-8116-3945fb915597',
      "status": "Not Executed",
      "scheduled_date": "2024-11-27",
      "charges": "No charges identified",
      "contact_made": True,
      "additional_info": "Billing clarification provided for another service order."
    },
    {
      "_id": "007910601880",
      "customer_id": 'f08a6894-1863-491d-8116-3945fb915597',
      "status": "Executed",
      "scheduled_date": "2024-12-08",
      "execution_date": "2024-12-08",
      "charges": "Charges incurred",
      "payment_plan": {
        "installments": 2,
        "confirmed": True,
        "installment_amount": 144.50
      },
      "communication_details": {
        "contact_type": "Phone",
        "request_details": "Detailed breakdown of charges and installment confirmation",
        "protocol_number": "8115209240"
      }
    }
  ]
response = collection.insert_many(orders)
print(response)

In [ ]:
document = collection.find_one()
print(document)